## Install libraries

In [ ]:
! pip install --quiet langchain langchain_cohere langchain_experimental

In [10]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.3 MB/s eta 0:00:00


In [95]:
!pip install gpt4all

In [1]:
# Create the Cohere chat model
from langchain_cohere.chat_models import ChatCohere
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

## Create a database and table for sample example tools

In [4]:
import sqlite3

conn = sqlite3.connect('customer_flight.db')

cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS flights (
    id INTEGER PRIMARY KEY,
    flight_no INT,
    arrive TEXT,
    take_off TEXT,
    destination TEXT,
    origin TEXT
)
"""

cursor.execute(create_table_query)
print("Table 'flights' created successfully")

sample_data = [
    (1, 101, '10:30', '12:00', 'تهران', 'اصفهان'),
    (2, 102, '14:45', '16:15', 'شیراز', 'مشهد'),
    (3, 103, '18:00', '20:30', 'اهواز', 'رشت'),
    (4, 104, '08:15', '10:00', 'یزد', 'قم'),
    (5, 105, '21:00', '22:30', 'کرمان', 'همدان')
]

cursor.executemany("INSERT INTO flights (id, flight_no, arrive, take_off, destination, origin) VALUES (?, ?, ?, ?, ?, ?)", sample_data)
print("Sample data inserted successfully")

conn.commit()
conn.close()


Table 'flights' created successfully
Sample data inserted successfully


In [5]:
import sqlite3
from langchain_core.tools import tool

@tool
def search_flights_by_destination(destination):
    """Search for flights based on destination. Values are Persian"""
    conn = sqlite3.connect('customer_flight.db')
    cursor = conn.cursor()

    query = "SELECT * FROM flights WHERE destination = ?"
    cursor.execute(query, (destination,))
    rows = cursor.fetchall()
    column_names = [column[0] for column in cursor.description]
    results = [dict(zip(column_names, row)) for row in rows]

    cursor.close()
    conn.close()

    return results


## Use FAQ in https://www.alibaba.ir/iranout for createing another tool

In [10]:
import pandas as pd

df = pd.read_excel('questions_and_answers.xlsx')

doc = df.to_dict('records')

print(doc)


[{'Question': 'بلیط پرواز چه کشورها و ایرلاین\u200cهایی را می\u200cتوانم\u200c در سایت علی\u200cبابا جستجو و خریداری کنم؟', 'Answer': 'بلیط تمام خطوط هوایی دنیا در سایت علی\u200cبابا موجود است، چه پروازهایی که مبدا یا مقصد آنها ایران است و چه پروازهای داخلی دورترین کشورهای دنیا. پروازهای ایرلاین\u200cهایی مثل لوفت\u200cهانزا، امارات، قطرایرویز، ترکیش\u200cایر، ایرفرانس، کی\u200cال\u200cام، آئروفلوت، آلیتالیا، اوکراینی، ایرایژیا، پگاسوس و ده\u200cها ایرلاین دیگر در علی\u200cبابا قابل تهیه هستند. همچنین بلیط پروازهای خارجیِ شرکت های هواپیمایی داخلی مانند ماهان، ایران\u200cایر، قشم ایر، آتا و .. نیز روی سایت علی\u200cبابا به فروش می\u200cرسد.'}, {'Question': 'چطور می توانم در مورد مقررات پرواز یا ویزا مربوط به سفرم اطمینان پیدا کنم؟', 'Answer': 'حتما قبل از انتخاب پرواز خود در مورد مقررات پرواز و قوانین مربوط به ویزا تحقیق کنید (مثلا ممکن است مقصد سفر شما نیاز به ویزا نداشته باشد ولی برای توقفی که در مسیر دارد نیاز به ویزای ترانزیت داشته باشید). برای این کار به صفحه قوانین و مقررات پرواز 

In [19]:
from langchain_community.embeddings import GPT4AllEmbeddings
import numpy as np
class VectorStoreRetriever:
    def __init__(self, docs: list, vectors: list, oai_client):
        self._arr = np.array(vectors)
        self._docs = docs
        self._client = oai_client

    @classmethod
    def from_docs(cls, docs, oai_client):
        embeddings = oai_client.embed_documents([doc["Question"] for doc in docs])
        vectors = [emb for emb in embeddings]
        return cls(docs, vectors, oai_client)

    def query(self, query: str, k: int = 5) -> list[dict]:
        embed = self._client.embed_documents([query])

        # "@" is just a matrix multiplication in python
        scores = np.array(embed) @ self._arr.T
        scores = scores[0]
        top_k_idx = np.argpartition(scores, -k)[-k:]
        top_k_idx_sorted = top_k_idx[np.argsort(-scores[top_k_idx])]



        return [
            {**self._docs[idx], "similarity": scores[idx]} for idx in top_k_idx_sorted
        ]


retriever = VectorStoreRetriever.from_docs(doc, GPT4AllEmbeddings())


@tool
def lookup_policy(query: str) -> str:
    """Looks up frequently asked questions (FAQ) in a Persian-language document.

    Parameters:
        query (str): The query or question for which the policy is being looked up.

    Returns:
        str: The policy or answer corresponding to the provided query."""
    docs = retriever.query(query, k=2)
    return "\n\n".join([doc["Answer"] for doc in docs])

## Define llm

In [20]:
os.environ['COHERE_API_KEY'] = ""
llm_model = ChatCohere(model="command-r-plus", temperature=0.3)

## Search on web is another tool


In [21]:
os.environ['TAVILY_API_KEY'] = ""
internet_search = TavilySearchResults()

## Create a Prompt template

In [27]:
# Create the prompt
prompt = ChatPromptTemplate.from_template("{input} Just search in Persian-language in SQLite for every response.")


# Create the ReAct agent
agent = create_cohere_react_agent(
    llm = llm_model,
    tools = [internet_search,lookup_policy, search_flights_by_destination],
    prompt=prompt,
)

In [28]:
agent_executor = AgentExecutor(agent = agent,
                               tools=[internet_search,lookup_policy, search_flights_by_destination],
                               verbose=True)

In [31]:
questions = {

    "input1":"سوالی هم دارم ارزان ترین بلیط رو چه طوری بخرم؟",
    "input2": "می خوام برم شیراز با چه هواپیمایی برم؟",
    "input3": "چه ساعتی هواپیمای کرمان حرکت میکنه؟"
}

for key, value in questions.items():
    # Invoke the agent with each question
    result = agent_executor.invoke({
        "input": value
    })
    print(result)




> Entering new AgentExecutor chain...

I will search for the cheapest way to buy a ticket in Persian.
{'tool_name': 'lookup_policy', 'parameters': {'query': 'ارزان ترین بلیط رو چه طوری بخرم'}}
ارزان بودن بلیط سفر به عوامل مختلفی مرتبط است که برخی از مهم ترین آنها این موارد هستند:
در پروازهای سیستمی معمولا هرچه به زمان پرواز نزدیک تر می شویم به دلیل پر شدن کلاس نرخی های ارزان تر، قیمت آن افزایش پیدا می کند. معمولا هرچه زودتر پروازتان را بخرید ارزان تر است. بلیط ایرلاین های مختلف را چک کنید. برای اغلب مسیرها ایرلاین های کم هزینه (Low Cost Airline) وجود دارند که در ازای خدمات و امکانات کمتر، پروازهای ارزان تری به شما پیشنهاد می دهند. در روزهای مختلف سال، فصل و هفته و حتی ساعات متفاوت، قیمت یک پرواز ممکن است تغییرات زیادی داشته باشد. با استفاده از تقویم قیمتی و فیلترهای تعبیه شده جستجو می توانید با چند روز جابجایی پروازهای به مراتب ارزان تری پیدا کنید. علاوه بر این، پروازهای توقف دار و صبح زود هم معمولا پروازهای ارزان تری هستند. بلیط خود را از سایت های معتبر خریداری کنید و قیمت آنها را د

## Resources:

- [GenAICourse YouTube Channel in Persian](https://www.youtube.com/@GenAICourse)
- [Data Analyst Agent Cohere and Langchain GitHub Notebook](https://github.com/cohere-ai/notebooks/blob/main/notebooks/Data_Analyst_Agent_Cohere_and_Langchain.ipynb?ref=cohere-ai.ghost.io)
- [Langgraph Tutorials: Customer Support](https://langchain-ai.github.io/langgraph/tutorials/customer-support/customer-support/)


### Crawling for FAQ

In [161]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.alibaba.ir/iranout'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

details = soup.find_all('details')

questions = []
answers = []

for detail in details:
    question = detail.find(class_='a-accordion__button')
    answer = detail.find(class_='faq-wrapper__description')

    if question and answer:
        questions.append(question.text.strip())
        answers.append(answer.text.strip())

df = pd.DataFrame({'Question': questions, 'Answer': answers})

df.to_excel('questions_and_answers.xlsx', index=False)
